# Load dataset

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense


# Load dataset
data = pd.read_csv('input/Dataset.csv')

# Process Labels

In [4]:
data


,sAddress,rAddress,sMACs,rMACs,sIPs,rIPs,protocol,startDate,endDate,start,...,sAckDelayMax,rAckDelayMax,sAckDelayMin,rAckDelayMin,sAckDelayAvg,rAckDelayAvg,IT_B_Label,IT_M_Label,NST_B_Label,NST_M_Label
0,192.168.0.11,192.168.0.21,02:42:c0:a8:00:0b,02:42:c0:a8:00:15,192.168.0.11,192.168.0.21,IPV4-TCP,2022-09-16 11:14:39.510337,2022-09-16 11:14:40.010330,1.663320e+09,...,0.001,0.497,0.0,0.0,0.000219,0.031154,0,Normal,0,Normal
1,192.168.0.12,192.168.0.21,02:42:c0:a8:00:0c,02:42:c0:a8:00:15,192.168.0.12,192.168.0.21,IPV4-TCP,2022-09-16 11:14:39.513421,2022-09-16 11:14:40.013394,1.663320e+09,...,0.000,0.497,0.0,0.0,0.000126,0.027727,0,Normal,0,Normal
2,192.168.0.11,192.168.0.12,02:42:c0:a8:00:0b,02:42:c0:a8:00:0c,192.168.0.11,192.168.0.12,IPV4-TCP,2022-09-16 11:14:39.601160,2022-09-16 11:14:40.054139,1.663320e+09,...,0.198,0.200,0.0,0.0,0.023117,0.030055,0,Normal,0,Normal
3,192.168.0.11,192.168.0.12,02:42:c0:a8:00:0b,02:42:c0:a8:00:0c,192.168.0.11,192.168.0.12,IPV4-TCP,2022-09-16 11:14:40.200719,2022-09-16 11:14:40.602902,1.663320e+09,...,0.197,0.200,0.0,0.0,0.020180,0.028530,0,Normal,0,Normal
4,192.168.0.11,192.168.0.21,02:42:c0:a8:00:0b,02:42:c0:a8:00:15,192.168.0.11,192.168.0.21,IPV4-TCP,2022-09-16 11:14:40.010348,2022-09-16 11:14:40.507654,1.663320e+09,...,0.000,0.493,0.0,0.0,0.000094,0.017102,0,Normal,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45713,192.168.0.22,192.168.0.41,02:42:c0:a8:00:16,02:42:c0:a8:00:29,192.168.0.22,192.168.0.41,IPV4-TCP,2022-09-16 13:23:30.589943,2022-09-16 13:23:30.877939,1.663327e+09,...,0.000,NaN,0.0,NaN,0.000007,NaN,1,port-scan,1,port-scan
45714,192.168.0.11,192.168.0.41,02:42:c0:a8:00:0b,02:42:c0:a8:00:29,192.168.0.11,192.168.0.41,IPV4-ICMP,2022-09-16 13:11:01.401338,2022-09-16 13:11:01.401338,1.663327e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,1,replay,1,replay
45715,192.168.0.12,192.168.0.41,02:42:c0:a8:00:0c,02:42:c0:a8:00:29,192.168.0.12,192.168.0.41,IPV4-ICMP,2022-09-16 13:11:01.401604,2022-09-16 13:11:01.401604,1.663327e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,1,replay,1,replay
45716,192.168.0.21,192.168.0.41,02:42:c0:a8:00:15,02:42:c0:a8:00:29,192.168.0.21,192.168.0.41,IPV4-ICMP,2022-09-16 13:11:03.511387,2022-09-16 13:11:03.511387,1.663327e+09,...,NaN,NaN,NaN,NaN,NaN,NaN,1,replay,1,replay


In [6]:
label_column = ['NST_M_Label']
label_columns_all = ['IT_B_Label', 'IT_M_Label', 'NST_B_Label', 'NST_M_Label']

# Encode labels to numerical values
label_encoder_y = LabelEncoder()
y_encoded = label_encoder_y.fit_transform(data[label_column])

# Convert labels to one-hot encoding
y_one_hot = pd.get_dummies(y_encoded).values

# Drop all label columns
data.drop(columns=label_columns_all)

D:\OneDrive - RISE\Code Repo\AnomalyDetection\Interpreter\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,sAddress,rAddress,sMACs,rMACs,sIPs,rIPs,protocol,startDate,endDate,start,...,sWinTCP,rWinTCP,sFragmentRate,rFragmentRate,sAckDelayMax,rAckDelayMax,sAckDelayMin,rAckDelayMin,sAckDelayAvg,rAckDelayAvg
0,192.168.0.11,192.168.0.21,02:42:c0:a8:00:0b,02:42:c0:a8:00:15,192.168.0.11,192.168.0.21,IPV4-TCP,2022-09-16 11:14:39.510337,2022-09-16 11:14:40.010330,1.663320e+09,...,509.00,502.0,0.0,0.0,0.001,0.497,0.0,0.0,0.000219,0.031154
1,192.168.0.12,192.168.0.21,02:42:c0:a8:00:0c,02:42:c0:a8:00:15,192.168.0.12,192.168.0.21,IPV4-TCP,2022-09-16 11:14:39.513421,2022-09-16 11:14:40.013394,1.663320e+09,...,509.00,502.0,0.0,0.0,0.000,0.497,0.0,0.0,0.000126,0.027727
2,192.168.0.11,192.168.0.12,02:42:c0:a8:00:0b,02:42:c0:a8:00:0c,192.168.0.11,192.168.0.12,IPV4-TCP,2022-09-16 11:14:39.601160,2022-09-16 11:14:40.054139,1.663320e+09,...,503.05,506.2,0.0,0.0,0.198,0.200,0.0,0.0,0.023117,0.030055
3,192.168.0.11,192.168.0.12,02:42:c0:a8:00:0b,02:42:c0:a8:00:0c,192.168.0.11,192.168.0.12,IPV4-TCP,2022-09-16 11:14:40.200719,2022-09-16 11:14:40.602902,1.663320e+09,...,503.00,506.2,0.0,0.0,0.197,0.200,0.0,0.0,0.020180,0.028530
4,192.168.0.11,192.168.0.21,02:42:c0:a8:00:0b,02:42:c0:a8:00:15,192.168.0.11,192.168.0.21,IPV4-TCP,2022-09-16 11:14:40.010348,2022-09-16 11:14:40.507654,1.663320e+09,...,509.00,502.0,0.0,0.0,0.000,0.493,0.0,0.0,0.000094,0.017102
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45713,192.168.0.22,192.168.0.41,02:42:c0:a8:00:16,02:42:c0:a8:00:29,192.168.0.22,192.168.0.41,IPV4-TCP,2022-09-16 13:23:30.589943,2022-09-16 13:23:30.877939,1.663327e+09,...,0.00,1024.0,0.0,0.0,0.000,NaN,0.0,NaN,0.000007,NaN
45714,192.168.0.11,192.168.0.41,02:42:c0:a8:00:0b,02:42:c0:a8:00:29,192.168.0.11,192.168.0.41,IPV4-ICMP,2022-09-16 13:11:01.401338,2022-09-16 13:11:01.401338,1.663327e+09,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
45715,192.168.0.12,192.168.0.41,02:42:c0:a8:00:0c,02:42:c0:a8:00:29,192.168.0.12,192.168.0.41,IPV4-ICMP,2022-09-16 13:11:01.401604,2022-09-16 13:11:01.401604,1.663327e+09,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
45716,192.168.0.21,192.168.0.41,02:42:c0:a8:00:15,02:42:c0:a8:00:29,192.168.0.21,192.168.0.41,IPV4-ICMP,2022-09-16 13:11:03.511387,2022-09-16 13:11:03.511387,1.663327e+09,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
from sklearn.compose import ColumnTransformer

class columnDropperTransformer():
    def __init__(self,columns):
        self.columns=columns

    def transform(self,X,y=None):
        return X.drop(self.columns,axis=1)

    def fit(self, X, y=None):
        return self

unused_columns = ['sAddress', 'rAddress', 'sMACs', 'rMACs', 'sIPs', 'rIPs', 'startDate', 'endDate', 'start', 'end', 'startOffset', 'endOffset']
categorical_columns = ['protocol']

# num_columns = data.select_dtypes(np.number).columns.tolist()
# cat_columns = data.select_dtypes('object').columns.tolist()

pipeline_categorial = ColumnTransformer(
    [
        ('ohe_encoder', OneHotEncoder(sparse_output=False, handle_unknown='ignore'),[0])
    ], remainder='passthrough'
    )

preprocess = Pipeline([
    ('dropper' , columnDropperTransformer(unused_columns)),
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('col_tra_2',pipeline_categorial),
    ('scaler', StandardScaler())
])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, y_one_hot, test_size=0.2, random_state=42, stratify=y_encoded)

# Preprocessing pipeline
X_train = preprocess.fit_transform(X_train)
X_test = preprocess.transform(X_test)


D:\OneDrive - RISE\Code Repo\AnomalyDetection\Interpreter\lib\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


# Simple Test

In [189]:
from tensorflow.python.estimator import keras
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from scikeras.wrappers import KerasClassifier

def create_model(input_dim, output_dim, hidden_layer_size=64, alpha=0.01):
    model = Sequential()
    model.add(Dense(hidden_layer_size, input_dim=input_dim, activation='relu'))
    model.add(Dense(output_dim, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(model=create_model, input_dim=X_train.shape[1], output_dim=y_train.shape[1], epochs=10, batch_size=32, verbose=0)


model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Inverse transform one-hot encoded predictions to original labels
# y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred_classes)
conf_matrix = confusion_matrix(y_test.argmax(axis=1), y_pred_classes)
print(f'Test Accuracy: {accuracy}')
print("Confusion Matrix:")
print(conf_matrix)

Test Accuracy: 0.977580927384077
Confusion Matrix:
[[7325    0    3    6    6    1]
 [   0  387    0    0    0    0]
 [   0    0   32    3    3    0]
 [   5    0   18  489    1    4]
 [  20    0   32   12  324    1]
 [  43    0    7   39    1  382]]


# Develop

In [3]:
from tensorflow.python.estimator import keras
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from scikeras.wrappers import KerasClassifier

def create_model(input_dim, output_dim, hidden_layer_size=64, activation='relu'):
    model = Sequential()
    model.add(Dense(hidden_layer_size, input_dim=input_dim, activation=activation))
    model.add(Dense(output_dim, activation='softmax'))
    # optimizer = keras.optimizers.Adam(learning_rate= alpha)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = KerasClassifier(model=create_model, input_dim=X_train.shape[1], output_dim=y_train.shape[1], epochs=10, batch_size=32, verbose=2)
param_grid = {
    'model__hidden_layer_size': [32, 64, 128],
    'model__activation': ['relu', 'tanh', 'sigmoid'],
    # 'model__alpha': [0.0001, 0.001, 0.01]
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=2)
grid_result = grid_search.fit(X_train, y_train)

print(f"Best Hyperparameters: {grid_result.best_params_}")
best_model = grid_result.best_estimator_


# Make predictions on the test set
y_pred = best_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Inverse transform one-hot encoded predictions to original labels
# y_pred_labels = label_encoder.inverse_transform(y_pred_classes)

# Evaluate the model on the test set
accuracy = accuracy_score(y_test.argmax(axis=1), y_pred_classes)
print(f'Test Accuracy: {accuracy}')

Fitting 3 folds for each of 9 candidates, totalling 27 fits
Epoch 1/10
762/762 - 2s - loss: 0.3565 - accuracy: 0.9012 - 2s/epoch - 2ms/step
Epoch 2/10
762/762 - 1s - loss: 0.1465 - accuracy: 0.9546 - 1s/epoch - 1ms/step
Epoch 3/10
762/762 - 1s - loss: 0.1128 - accuracy: 0.9634 - 1s/epoch - 1ms/step
Epoch 4/10
762/762 - 1s - loss: 0.0944 - accuracy: 0.9657 - 1s/epoch - 2ms/step
Epoch 5/10
762/762 - 1s - loss: 0.0818 - accuracy: 0.9697 - 1s/epoch - 1ms/step
Epoch 6/10
762/762 - 1s - loss: 0.0736 - accuracy: 0.9751 - 999ms/epoch - 1ms/step
Epoch 7/10
762/762 - 1s - loss: 0.0679 - accuracy: 0.9768 - 1s/epoch - 1ms/step
Epoch 8/10
762/762 - 1s - loss: 0.0638 - accuracy: 0.9769 - 1s/epoch - 2ms/step
Epoch 9/10
762/762 - 1s - loss: 0.0610 - accuracy: 0.9776 - 1s/epoch - 1ms/step
Epoch 10/10
762/762 - 1s - loss: 0.0590 - accuracy: 0.9779 - 1s/epoch - 1ms/step
381/381 - 1s - 507ms/epoch - 1ms/step
[CV] END model__activation=relu, model__hidden_layer_size=32; total time=  12.5s
Epoch 1/10
762/76

In [5]:
import joblib
from keras.models import load_model

# Assuming 'model' is your Keras model
joblib.dump(best_model, 'output/keras_classifier_model.joblib')
predict_pipeline = Pipeline([
    ('col_tra_2',preprocess),
    ('model' , best_model)
])
joblib.dump(predict_pipeline, 'output/predict.joblib')


INFO:tensorflow:Assets written to: C:\Users\ALIREZ~1\AppData\Local\Temp\tmpxkjd9ogc\assets


INFO:tensorflow:Assets written to: C:\Users\ALIREZ~1\AppData\Local\Temp\tmpxkjd9ogc\assets


INFO:tensorflow:Assets written to: C:\Users\ALIREZ~1\AppData\Local\Temp\tmpflwify18\assets


INFO:tensorflow:Assets written to: C:\Users\ALIREZ~1\AppData\Local\Temp\tmpflwify18\assets


['output/predict.joblib']